# Code

First, import the libraries and set the database path. 

In [1]:
# library import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from datetime import datetime

In [2]:
oil = pd.read_csv('oil.csv')
sample_submission = pd.read_csv('sample_submission.csv')
holidays = pd.read_csv('holidays_events.csv')
stores = pd.read_csv('stores.csv')
train_set = pd.read_csv('train.csv')
submission_set = pd.read_csv('test.csv')
transactions = pd.read_csv('transactions.csv')

In [3]:
def min_mag(x):
    k = int(np.argmin(np.abs(x)))
    
    try:
        y = list(x)[k]
    except Exception as e:
        print(f'x = {x} ({type(x)}), k = {k} ({type(k)})')
        raise e
        
    return y

# Boil down holidays to a more manageable set
holidays['date'] = pd.to_datetime(holidays['date'])

special_days = pd.concat([
    pd.merge(
        stores,
        holidays,
        right_on='locale_name',
        left_on='city',
        suffixes=('_store','_celebration')
    ),
    pd.merge(
        stores,
        holidays,
        right_on='locale_name',
        left_on='state',
        suffixes=('_store','_celebration')
    ),
    pd.merge(
        stores,
        holidays[holidays.locale == 'National'],
        how='cross',
        suffixes=('_store','_celebration')
    )
]).reset_index(drop=True)

special_days[['occasion','relative']] = pd.DataFrame(special_days.description.str.split('^(\D+)\s*([+-]\d+)?$').str[1:3].tolist())
special_days['relative'] = special_days['relative'].fillna('0').astype(int)

special_days['occasion'] = special_days['occasion'].str.replace('^Traslado\s+(.*)$', '\\1', regex=True)

special_days['occasion_cat'] = special_days['occasion'].str.lower().str.replace(
    '.*(?:fundacion|independencia|provincializacion|batalla).*', 'political', regex=True
).str.replace(
    '.*(?:cantonizacion).*','cantonizacion', regex=True
).str.replace(
    '^recupero puente\s+(.*)$','\\1', regex=True
).str.replace(
    '^puente\s+(.*)$','\\1', regex=True
).str.replace(
    '.*(?:futbol).*', 'futbol', regex=True
).str.replace(
    '.*(?:black friday|cyber monday).*', 'shopping', regex=True
).str.replace(
    '.*(?:terremoto manabi).*', 'earthquake', regex=True
).str.replace(
    '.*(?:dia del trabajo|dia de la madre|dia de difuntos|viernes santo).*', 'other', regex=True
)

special_days = special_days.loc[:,[
    'store_nbr', 'date', 'type_celebration', 'locale', 'occasion_cat', 'relative'
]]

special_days = pd.merge(
    special_days,
    pd.get_dummies(special_days['type_celebration']),
    left_index=True,
    right_index=True
).merge(
    pd.get_dummies(special_days['occasion_cat']),
    left_index=True,
    right_index=True
).merge(
    pd.get_dummies(special_days['locale']),
    left_index=True,
    right_index=True
).drop(
    columns=['type_celebration','occasion_cat','locale']
)

# handle multiple holidays on the same day
special_days = special_days.groupby(
    ['store_nbr', 'date']
).agg({
    'relative': min_mag,
    'Additional': 'max',
    'Bridge': 'max',
    'Event': 'max',
    'Holiday': 'max',
    'Transfer': 'max',
    'Work Day': 'min',
    'cantonizacion': 'max',
    'carnaval': 'max',
    'earthquake': 'max',
    'futbol': 'max',
    'navidad': 'max',
    'other': 'max',
    'political': 'max',
    'primer dia del ano': 'max',
    'shopping': 'max',
    'Local': 'max',
    'National': 'max',
    'Regional': 'max',
})

In [4]:
# additional info
mean_price = 73.694969
std_price = 26.581630

oil_clean = oil
oil_clean['date'] = pd.to_datetime(oil_clean['date'])

start_date = oil_clean.date.min()
end_date = oil_clean.date.max()
desired_date_range = pd.date_range(start=start_date, end=end_date, freq='D')

# Reindex the DataFrame with the desired date range
oil_clean = oil_clean.set_index('date').reindex(desired_date_range).reset_index().rename(columns={'index': 'date'})
oil_clean['oil_price'] = oil_clean.pop('dcoilwtico')
oil_clean['oil_price'] = oil_clean['oil_price'].fillna(method='ffill')
oil_clean['oil_price'] = oil_clean['oil_price'].fillna(method='bfill')# this should just drop Jan 1 of first year
oil_clean['oil_price'] = ( oil_clean['oil_price'] - mean_price ) / std_price

store_type = pd.get_dummies(stores['type'])
store_type = store_type.rename(columns={col: f'store_type_{col}' for col in store_type.columns})

store_cluster = pd.get_dummies(stores['cluster'])
store_cluster = store_cluster.rename(columns={col: f'store_cluster_{col}' for col in store_cluster.columns})

stores_categorization = stores.merge(
    store_type,
    left_index=True,
    right_index=True
).merge(
    store_cluster,
    left_index=True,
    right_index=True
).drop(
    columns=['city', 'state', 'type', 'cluster']
)

In [5]:
# get sales (dependent variable), and promotions
def variable_extraction(
    dataset
):
    dataset['date'] = pd.to_datetime(dataset['date'])
    
    predictors = dataset.pivot_table(
        index=['date','store_nbr'],
        columns='family',
        values='onpromotion',
        aggfunc='mean'
    )
    
    if 'sales' in dataset:
        sales = dataset.pivot_table(
            index=['date','store_nbr'],
            columns='family',
            values='sales',
            aggfunc='mean'
        ).sort_index()
        
        #no_store = (sales == 0).all(axis=1).groupby('store_nbr').cummin()
        
        # remove data for period before store exists
        # sales = sales.loc[~no_store, :]
        # predictors = predictors.loc[~no_store, :]
    else:
        sales = None
    
    predictors = pd.merge(
        predictors,
        special_days,
        on=['date','store_nbr'],
        how='left'
    ).fillna({
        'relative': 0,
        'Work Day': True,
    }).fillna({key: False for key in [
        'Additional','Bridge','Event',
        'Holiday','Transfer',
        'cantonizacion', 'carnaval', 'earthquake',
        'futbol', 'navidad', 'other', 'political',
        'primer dia del ano', 'shopping',
        'Local', 'National', 'Regional'
    ]}
    ).reset_index()
        
    predictors = predictors.merge(
        oil_clean,
        on='date',
        how='left'
    ).merge(
        stores_categorization,
        on='store_nbr',
        how='left'
    ).sort_values([
        'store_nbr', 'date'
    ]).reset_index(drop=True)
        
    # seasonality
    d = pd.concat([
        (predictors.date.dt.day_of_week).rename('day_of_week'),
        pd.Series(
            np.where(
                predictors.date.dt.day <= 15,
                15 - predictors.date.dt.day,
                predictors.date.dt.days_in_month - predictors.date.dt.day
            )
        ).rename('semimonthly'),
        predictors.date.dt.day.rename('day_of_month'),
        predictors.date.map(quarter_part).rename('quarterly'),
        predictors.date.dt.month.rename('month'),
    ], axis=1)
    
    #cosD = np.cos(d).rename(columns={col: 'cos_' + col for col in d.columns})
    #sinD = np.sin(d).rename(columns={col: 'sin_' + col for col in d.columns})
    
    predictors = pd.concat([
        predictors,
        #cosD,
        #sinD,
        d
    ], axis=1)
    
    store_OHE = pd.get_dummies(predictors.store_nbr)
    store_OHE = store_OHE.rename(columns={col: f'store_{col}' for col in store_OHE.columns})
        
    predictors = pd.merge(
        predictors,
        store_OHE,
        left_index=True,
        right_index=True,
        how='left'
    )
    
    predictors = predictors.set_index(['date','store_nbr']).sort_index()
    
    return (predictors, sales)

In [6]:
# data import
train_df, sales = variable_extraction(train_set)

Let's create and train our model. In this step, I will use a RandomForestRegressor.

In [7]:
# Model training with RandomForest
X = train_df.astype(float) #.drop(['sales', 'date', 'onpromotion', 'store_nbr'], axis=1)
y = sales

# Splitting data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [18]:
# Creating and training the RandomForest model
model = RandomForestRegressor(n_estimators=50, max_depth=40, n_jobs=-1, random_state=42)

RandomForestRegressor(max_depth=40, n_estimators=75, n_jobs=-1, random_state=42)

Check the model quality using RMSLE.

In [39]:
#train
model.fit(X_train, y_train)

# prediction
y_pred = model.predict(X_test)

# RMSLE
log_actual = np.log1p(y_test)
log_pred = np.log1p(y_pred)

rmsle = np.sqrt(np.mean(np.mean((log_pred - log_actual) ** 2, axis=0)))

print("RMSLE:", rmsle)

RMSLE: 1.3580584896842083


/tmp/ipykernel_1200349/2614005188.py:9: RuntimeWarning: invalid value encountered in log1p
  log_pred = np.log1p(y_pred)


Finally, create the file for the submission using test data.

In [12]:
submission_df, _ = variable_extraction(submission_set)

In [13]:
y_pred_submit = pd.DataFrame(
    model.predict(submission_df.astype(float)),
    index=submission_df.index,
    columns=sales.columns
)

submission_results = pd.merge(
    submission_set.drop(columns=['onpromotion']),
    pd.melt(
        y_pred_submit,
        ignore_index = False,
        value_name='sales'
    ).reset_index(),
    how='outer'
).sort_values('id').loc[
    :,
    ['id', 'sales']    
]

submission_results.to_csv('submission.csv', index=False)